In [1]:
from peft import AutoPeftModelForCausalLM, PeftModelForCausalLM, get_peft_config
from transformers import AutoTokenizer, LlamaTokenizer, AutoModelForCausalLM
from multihead_models import MultiHeadPeftModelForCausalLM
import torch
import json
import os

In [2]:
tokenizer = LlamaTokenizer.from_pretrained('/mnt/data/zoo/llama2/llama2-7b-hf/')
inp = tokenizer(
    ['There is a Ubuntu server visible at IP 43.205.13.243, port 22, offering the service cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5.\n',], 
    return_tensors="pt",
)  # Batch size 1
inp = {x:inp[x].cuda() for x in inp}

step=60
output_dir = f'/mnt/data/sonia/ckpts/sent3/checkpoint-{step}/'
print(output_dir)

/mnt/data/sonia/ckpts/sent3/checkpoint-60/


In [3]:
# model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)

`model.base_model.model.model` stays unchanged, while `model.base_model.model.lm_head` gets duplicated up to however many heads we want. For each head, apply some mask that will only predict tokens from that head's vocabulary

In [4]:
# class MultiHeadPeftModelForCausalLM(PeftModelForCausalLM):
#     def __init__(self, config, model, heads, head_masks):
#         super().__init__(model, config)
#         self.config = config
#         self.model = model
#         self.heads = heads
#         self.head_masks = head_masks
    
#     def from_one_head(peft_model, head_masks, head_max_tokens, *args, **kwargs):
#         """peft_model: A PeftModelForCausalLM,
#             head_masks: a list of masks for head output vocabulary,
#             head_max_tokens: a list of ints, where the nth element is the 
#                         max number of new tokens to be generated by the nth head"""
#         model = list(peft_model.model.children())[0]
#         orighead = list(peft_model.model.children())[1]
#         mhmodel = MultiHeadPeftModelForCausalLM(peft_model.peft_config['default'], model, [], [])
#         assert len(head_masks) == len(head_max_tokens)
#         for i in range(len(head_masks)):
#             mhmodel._add_head(orighead)
#         return mhmodel
    
#     def _add_head(self, orighead):
#         """in features and out features the same, but a mask will be applied at prediction.
#         Basically just a method to deep-copy the original head"""
#         in_features, out_features, bias = orighead.in_features, orighead.out_features, orighead.bias
#         newhead = type(orighead)(in_features, out_features, bias)
#         newhead.load_state_dict(orighead.state_dict())
#         self.heads.append(newhead)
        
#     def generate(self, *args, **kwargs):
#         self.base_model.prepare_inputs_for_generation = self.prepare_inputs_for_generation
#         if hasattr(self.base_model, "model"):
#             self.base_model.model.generation_config = self.generation_config
#         else:
#             self.base_model.generation_config = self.generation_config
            
#         print(args, kwargs)
#         outputs = self.base_model.forward(*args)

# peft_config = get_peft_config(config)
ohmodel = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
mhmodel = MultiHeadPeftModelForCausalLM.from_one_head(ohmodel, [torch.ones(32000)], [1])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
outputs = ohmodel.generate(**inp, max_new_tokens=58)
out=tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(out[0])

/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


There is a Ubuntu server visible at IP 43.205.13.243, port 22, offering the service cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5.
There is a Ubuntu server visible at IP 43.205.13.243, port 22, offering the service cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5.


In [19]:
mhmodel.base_model.named_modules()

<generator object Module.named_modules at 0x7fd5c0fea0e0>

In [ ]:
inter = mhmodel.model.generate(**inp)
outputs = mhmodel.heads[0](inter.last_hidden_state.cpu().float())
# outputs.shape, outputs 
outmax = outputs.squeeze().argmax(axis=1)
tokenizer.batch_decode([outmax], skip_special_tokens=True, clean_up_tokenization_spaces=False)

TypeError: The current model class (LlamaModel) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'LlamaForCausalLM'}

In [9]:
ohmodel.base_model.model.generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.31.0"
}